In [34]:
import json
import pandas as pd
import os


In [19]:
# Define your cities
cities = ["San_Antonio", "Austin", "Houston"]
state = "TX"

# Load all properties
all_properties = []

for city in cities:
    file_path = f"data/raw/property_data_{city}_{state}.json"
    
    print(f"Loading {city}...")
    with open(file_path, "r") as f:
        data = json.load(f)
    
    # Add city identifier to each property
    for prop in data:
        prop['source_city'] = city
        prop['source_state'] = state
        all_properties.append(prop)
    
    print(f"✓ Loaded {len(data)} properties from {city}")

# Create DataFrame
df = pd.DataFrame(all_properties)

print(f"\n{'='*50}")
print(f"Total properties loaded: {len(df)}")
print(f"Total columns: {len(df.columns)}")
print(f"\nDataFrame shape: {df.shape}")
print(f"\nFirst few rows:")
print(df.head())

# View column names
print(f"\nAvailable columns:")
print(df.columns.tolist())

Loading San_Antonio...
✓ Loaded 50 properties from San_Antonio
Loading Austin...
✓ Loaded 50 properties from Austin
Loading Houston...
✓ Loaded 50 properties from Houston

Total properties loaded: 150
Total columns: 28

DataFrame shape: (150, 28)

First few rows:
                                            id  \
0         5509-Roanwood,-San-Antonio,-TX-78244   
1         5528-Lochmoor,-San-Antonio,-TX-78244   
2  5523-Lochmoor,-Apt-2,-San-Antonio,-TX-78244   
3         5721-Lochmoor,-San-Antonio,-TX-78244   
4  5519-Lochmoor,-Apt-2,-San-Antonio,-TX-78244   

                              formattedAddress   addressLine1 addressLine2  \
0         5509 Roanwood, San Antonio, TX 78244  5509 Roanwood         None   
1         5528 Lochmoor, San Antonio, TX 78244  5528 Lochmoor         None   
2  5523 Lochmoor, Apt 2, San Antonio, TX 78244  5523 Lochmoor        Apt 2   
3         5721 Lochmoor, San Antonio, TX 78244  5721 Lochmoor         None   
4  5519 Lochmoor, Apt 2, San Antonio, TX 7824

In [22]:
df = pd.json_normalize(all_properties)
print(df.head(2))

                                     id                      formattedAddress  \
0  5509-Roanwood,-San-Antonio,-TX-78244  5509 Roanwood, San Antonio, TX 78244   
1  5528-Lochmoor,-San-Antonio,-TX-78244  5528 Lochmoor, San Antonio, TX 78244   

    addressLine1 addressLine2         city state stateFips zipCode county  \
0  5509 Roanwood         None  San Antonio    TX        48   78244  Bexar   
1  5528 Lochmoor         None  San Antonio    TX        48   78244  Bexar   

  countyFips  ...  taxAssessments.2025.year  taxAssessments.2025.value  \
0        029  ...                       NaN                        NaN   
1        029  ...                       NaN                        NaN   

  taxAssessments.2025.land  taxAssessments.2025.improvements  \
0                      NaN                               NaN   
1                      NaN                               NaN   

   propertyTaxes.2013.year  propertyTaxes.2013.total  propertyTaxes.2014.year  \
0                      NaN 

In [25]:
for city in ["San_Antonio", "Austin", "Houston"]:
    city_df = df[df['source_city'] == city]
    print(f"\n{city} - Columns with non-null values:")
    print(city_df.columns[city_df.notna().any()].tolist())


San_Antonio - Columns with non-null values:
['id', 'formattedAddress', 'addressLine1', 'addressLine2', 'city', 'state', 'stateFips', 'zipCode', 'county', 'countyFips', 'latitude', 'longitude', 'propertyType', 'bedrooms', 'bathrooms', 'squareFootage', 'yearBuilt', 'source_city', 'source_state', 'lotSize', 'features.floorCount', 'assessorID', 'legalDescription', 'subdivision', 'ownerOccupied', 'features.architectureType', 'features.cooling', 'features.coolingType', 'features.exteriorType', 'features.fireplace', 'features.foundationType', 'features.garage', 'features.garageSpaces', 'features.garageType', 'features.heating', 'features.heatingType', 'features.roofType', 'features.roomCount', 'features.unitCount', 'taxAssessments.2015.year', 'taxAssessments.2015.value', 'taxAssessments.2015.land', 'taxAssessments.2015.improvements', 'taxAssessments.2016.year', 'taxAssessments.2016.value', 'taxAssessments.2016.land', 'taxAssessments.2016.improvements', 'taxAssessments.2017.year', 'taxAssessm

In [26]:
# Keep only essential columns
essential_columns = [
    'id', 'formattedAddress', 'city', 'state', 'zipCode', 
    'latitude', 'longitude', 'propertyType', 'bedrooms', 
    'bathrooms', 'squareFootage', 'lotSize', 'yearBuilt',
    'taxAssessments.2024.value', 'propertyTaxes.2024.total',
    'owner.names', 'ownerOccupied', 'features.garage', 
    'features.pool', 'source_city'
]

# Filter to existing columns
existing_cols = [col for col in essential_columns if col in df.columns]
df_clean = df[existing_cols].copy()

In [27]:
# Rename columns to snake_case
df_clean = df_clean.rename(columns={
    'id': 'property_id',
    'formattedAddress': 'formatted_address',
    'city': 'city',
    'state': 'state',
    'zipCode': 'zip_code',
    'latitude': 'latitude',
    'longitude': 'longitude',
    'propertyType': 'property_type',
    'bedrooms': 'bedrooms',
    'bathrooms': 'bathrooms',
    'squareFootage': 'square_footage',
    'lotSize': 'lot_size',
    'yearBuilt': 'year_built',
    'taxAssessments.2024.value': 'tax_assessment_2024',
    'propertyTaxes.2024.total': 'property_tax_2024',
    'owner.names': 'owner_name',
    'ownerOccupied': 'owner_occupied',
    'features.garage': 'has_garage',
    'features.pool': 'has_pool',
    'source_city': 'source_city'
})

print("Column names cleaned!")
print(df_clean.columns.tolist())

Column names cleaned!
['property_id', 'formatted_address', 'city', 'state', 'zip_code', 'latitude', 'longitude', 'property_type', 'bedrooms', 'bathrooms', 'square_footage', 'lot_size', 'year_built', 'tax_assessment_2024', 'property_tax_2024', 'owner_name', 'owner_occupied', 'has_garage', 'has_pool', 'source_city']


In [28]:
df

,id,formattedAddress,addressLine1,addressLine2,city,state,stateFips,zipCode,county,countyFips,...,taxAssessments.2025.year,taxAssessments.2025.value,taxAssessments.2025.land,taxAssessments.2025.improvements,propertyTaxes.2013.year,propertyTaxes.2013.total,propertyTaxes.2014.year,propertyTaxes.2014.total,propertyTaxes.2025.year,propertyTaxes.2025.total
0,"5509-Roanwood,-San-Antonio,-TX-78244","5509 Roanwood, San Antonio, TX 78244",5509 Roanwood,None,San Antonio,TX,48,78244,Bexar,029,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"5528-Lochmoor,-San-Antonio,-TX-78244","5528 Lochmoor, San Antonio, TX 78244",5528 Lochmoor,None,San Antonio,TX,48,78244,Bexar,029,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"5523-Lochmoor,-Apt-2,-San-Antonio,-TX-78244","5523 Lochmoor, Apt 2, San Antonio, TX 78244",5523 Lochmoor,Apt 2,San Antonio,TX,48,78244,Bexar,029,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"5721-Lochmoor,-San-Antonio,-TX-78244","5721 Lochmoor, San Antonio, TX 78244",5721 Lochmoor,None,San Antonio,TX,48,78244,Bexar,029,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,"5519-Lochmoor,-Apt-2,-San-Antonio,-TX-78244","5519 Lochmoor, Apt 2, San Antonio, TX 78244",5519 Lochmoor,Apt 2,San Antonio,TX,48,78244,Bexar,029,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145,"3833-Cummins-St,-Houston,-TX-77027","3833 Cummins St, Houston, TX 77027",3833 Cummins St,None,Houston,TX,48,77027,Harris,201,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
146,"10070-Westpark-Dr,-Houston,-TX-77042","10070 Westpark Dr, Houston, TX 77042",10070 Westpark Dr,None,Houston,TX,48,77042,Harris,201,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
147,"14504-Briar-Forest-Dr,-Houston,-TX-77077","14504 Briar Forest Dr, Houston, TX 77077",14504 Briar Forest Dr,None,Houston,TX,48,77077,Harris,201,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
148,"13155-Woodforest-Blvd,-Houston,-TX-77015","13155 Woodforest Blvd, Houston, TX 77015",13155 Woodforest Blvd,None,Houston,TX,48,77015,Harris,201,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [35]:
# Create directory structure
os.makedirs("data/transformed", exist_ok=True)

# Save cleaned data
df_clean.to_csv("data/transformed/all_properties_cleaned.csv", index=False)
print("✓ Saved cleaned data")

✓ Saved cleaned data
